# 所有策略使用範例

本 Notebook 展示 MediaOverload 專案中所有生成策略的使用方法。

## 支援的策略

1. **Text2ImageStrategy** - 文生圖
2. **Image2ImageStrategy** - 圖生圖
3. **Text2Image2ImageStrategy** - 文生圖 → 圖生圖
4. **Text2VideoStrategy** - 文生影片
5. **Text2Image2VideoStrategy** - 文生圖 → 圖生影片
6. **Text2LongVideoStrategy** - 文生長影片（尾幀驅動）
7. **Text2LongVideoFirstFrameStrategy** - 文生長影片（首幀驅動）
8. **StickerPackStrategy** - 貼圖包生成

## 使用模式

每個策略都支援兩種使用模式：
- **自定義 Prompt**：直接指定關鍵詞或描述
- **從資料庫獲取 News**：自動從資料庫獲取新聞作為生成素材

## 批量生成

所有策略都支援批量生成，可指定生成數量（如 30 張、50 張等）。


## 環境配置

### ComfyUI 連接設定

在 Windows 環境中運行時，需要設定環境變數：


In [1]:
import sys
import os
from pathlib import Path
from datetime import datetime

# 確保可以導入專案模組
project_root = Path.cwd().parent if Path.cwd().name == 'examples' else Path.cwd()
sys.path.insert(0, str(project_root))

# 設定 ComfyUI 連接環境變數（Windows 環境使用 localhost）
os.environ['COMFYUI_HOST'] = '127.0.0.1'
os.environ['COMFYUI_PORT'] = '8188'

# 導入必要模組
from examples.quick_draw.helpers import FlexibleGenerator, ConfigBuilder
from examples.simple_content_service import SimpleContentGenerationService
from lib.repositories.news_repository import NewsRepository
from lib.repositories.character_repository import CharacterRepository
from lib.database import db_pool
from lib.media_auto.models.vision.vision_manager import VisionManagerBuilder
from dotenv import load_dotenv
from IPython.display import Image, display
import glob

# 載入環境變數
env_path = project_root / 'media_overload.env'
load_dotenv(env_path)

# 初始化資料庫連接
db_pool.initialize('mysql',
                  host=os.environ['mysql_host'],
                  port=int(os.environ['mysql_port']),
                  user=os.environ['mysql_user'],
                  password=os.environ['mysql_password'],
                  db_name=os.environ['mysql_db_name'])

mysql_conn = db_pool.get_connection('mysql')

# 初始化服務
character_repository = CharacterRepository(mysql_conn)
news_repository = NewsRepository(mysql_conn)
vision_manager = VisionManagerBuilder() \
    .with_vision_model('openrouter') \
    .with_text_model('openrouter') \
    .with_random_models(True) \
    .build()

content_service = SimpleContentGenerationService(
    character_repository=character_repository,
    vision_manager=vision_manager
)

generator = FlexibleGenerator(verbose=True)

print("✓ 環境初始化完成")
print(f"✓ ComfyUI 連接設定: {os.environ['COMFYUI_HOST']}:{os.environ['COMFYUI_PORT']}")

2025-12-07 23:04:09,676 - mediaoverload - INFO - 隨機選擇的 Vision 模型: google/gemini-2.0-flash-exp:free
2025-12-07 23:04:09,681 - mediaoverload - INFO - 隨機選擇的 Text 模型: deepseek/deepseek-r1-0528:free
2025-12-07 23:04:09,690 - mediaoverload - INFO - 隨機選擇的 Vision 模型: google/gemma-3-27b-it:free
2025-12-07 23:04:09,691 - mediaoverload - INFO - 隨機選擇的 Text 模型: z-ai/glm-4.5-air:free


正在載入環境變數: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\media_overload.env
環境變數載入成功
✓ 服務初始化完成
✓ 環境初始化完成
✓ ComfyUI 連接設定: 127.0.0.1:8188


## 輔助函數

定義用於獲取新聞和批量生成的輔助函數：


In [ ]:
def get_news_from_db(days_back=7):
    """從資料庫獲取新聞
    
    Args:
        days_back: 回溯天數，預設 7 天
        
    Returns:
        包含 title 和 keyword 的字典，如果沒有則返回 None
    """
    date_filter = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
    news = news_repository.get_random_news(date_filter)
    
    if news:
        print(f"📰 獲取到新聞:")
        print(f"   標題: {news['title']}")
        print(f"   關鍵詞: {news['keyword']}")
        return news
    else:
        print("⚠️ 未找到符合條件的新聞")
        return None

def batch_generate_by_count(strategy_type, num_total, 
                           use_news=False, 
                           custom_keywords=None,
                           character="kirby",
                           system_prompt="stable_diffusion_prompt",
                           **kwargs):
    """批量生成指定數量的媒體
    
    Args:
        strategy_type: 策略類型 ('text2image', 'text2video', 等)
        num_total: 總共要生成的數量
        use_news: 是否使用新聞（True）或自定義關鍵詞（False）
        custom_keywords: 自定義關鍵詞（當 use_news=False 時使用）
        character: 角色名稱
        system_prompt: 系統提示詞
        **kwargs: 其他策略特定參數
        
    Returns:
        生成結果列表
    """
    results = []
    
    # 計算需要生成幾輪（每輪生成數量由策略決定）
    if strategy_type in ['text2image', 'text2img']:
        images_per_batch = kwargs.get('num_images', 4)
        num_batches = (num_total + images_per_batch - 1) // images_per_batch
    elif strategy_type in ['text2video', 't2v']:
        videos_per_batch = kwargs.get('num_videos', 2)
        num_batches = (num_total + videos_per_batch - 1) // videos_per_batch
    else:
        num_batches = 1
    
    print(f"\n📦 開始批量生成")
    print(f"   策略: {strategy_type}")
    print(f"   目標數量: {num_total}")
    print(f"   批次數: {num_batches}")
    print(f"   模式: {'從資料庫獲取新聞' if use_news else '自定義關鍵詞'}")
    print("="*60)
    
    for i in range(num_batches):
        print(f"\n[{i+1}/{num_batches}] 處理中...")
        
        # 獲取關鍵詞
        if use_news:
            news = get_news_from_db()
            if not news:
                print("⚠️ 無法獲取新聞，跳過此批次")
                continue
            keywords = news['keyword'] or news['title']
        else:
            keywords = custom_keywords or "peaceful scene, beautiful landscape"
        
        # 根據策略類型生成
        try:
            if strategy_type in ['text2image', 'text2img']:
                result = generator.generate_images(
                    keywords=keywords,
                    system_prompt=system_prompt,
                    character=character,
                    num_images=kwargs.get('num_images', 4),
                    output_subdir=f'batch_{strategy_type}_{i+1}',
                    **{k: v for k, v in kwargs.items() if k != 'num_images'}
                )
            elif strategy_type in ['text2video', 't2v']:
                result = generator.generate_videos(
                    keywords=keywords,
                    system_prompt=system_prompt,
                    character=character,
                    num_videos=kwargs.get('num_videos', 2),
                    output_subdir=f'batch_{strategy_type}_{i+1}',
                    **{k: v for k, v in kwargs.items() if k != 'num_videos'}
                )
            else:
                # 其他策略需要使用 ConfigBuilder
                config = build_config_for_strategy(
                    strategy_type, keywords, character, 
                    system_prompt, **kwargs
                )
                result = content_service.generate_content(config)
            
            results.append({
                'batch': i+1,
                'keywords': keywords,
                'result': result
            })
            
            print(f"✅ 批次 {i+1} 完成，生成 {len(result.get('media_files', []))} 個檔案")
            
        except Exception as e:
            print(f"❌ 批次 {i+1} 失敗: {str(e)}")
            continue
    
    print("\n" + "="*60)
    total_files = sum(len(r['result'].get('media_files', [])) for r in results)
    print(f"✅ 批量生成完成！總共生成: {total_files} 個檔案")
    
    return results

def build_config_for_strategy(strategy_type, keywords, character, 
                              system_prompt, **kwargs):
    """為特定策略建立配置"""
    workflow_folder = project_root / 'configs' / 'workflow'
    output_dir = project_root / 'output_media' / f'{strategy_type}_output'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    builder = ConfigBuilder() \
        .with_character(character) \
        .with_prompt(keywords) \
        .with_output_dir(str(output_dir)) \
        .with_image_system_prompt(system_prompt) \
        .with_generation_type(strategy_type)
    
    # 策略特定配置
    if strategy_type == 'image2image':
        builder.with_input_image(kwargs.get('input_image_path', ''))
        builder.with_denoise(kwargs.get('denoise', 0.6))
    elif strategy_type == 'text2image2image':
        builder.with_additional_params(**{
            'strategies': {
                'text2image2image': {
                    'first_stage': {
                        'images_per_description': kwargs.get('num_images', 4),
                        'workflow_path': str(workflow_folder / 'nova-anime-xl.json')
                    },
                    'second_stage': {
                        'images_per_input': kwargs.get('images_per_input', 2),
                        'workflow_path': str(workflow_folder / 'image_to_image.json')
                    }
                }
            }
        })
    elif strategy_type == 'text2image2video':
        builder.with_additional_params(**{
            'strategies': {
                'text2image2video': {
                    'first_stage': {
                        'images_per_description': kwargs.get('num_images', 1),
                        't2i_workflow_path': str(workflow_folder / 'nova-anime-xl.json')
                    },
                    'video': {
                        'videos_per_image': kwargs.get('num_videos_per_image', 1),
                        'i2v_workflow_path': str(workflow_folder / 'wan2.2_gguf_i2v_audio.json')
                    }
                }
            }
        })
    elif strategy_type == 'sticker_pack':
        builder.with_additional_params(**{
            'strategies': {
                'sticker_pack': {
                    'expressions_count': kwargs.get('expressions_count', 10),
                    'animated_config': {
                        'enabled': kwargs.get('animated_enabled', True)
                    }
                }
            }
        })
    
    # 設定工作流
    default_workflow = kwargs.get('workflow', 'nova-anime-xl.json')
    builder.with_workflow(str(workflow_folder / default_workflow))
    
    return builder.build()

from datetime import timedelta

print("✓ 輔助函數定義完成")


✓ 輔助函數定義完成


# 1. Text2ImageStrategy - 文生圖

最基本的策略，從文字描述生成圖片。


## 1.1 自定義 Prompt 範例


In [ ]:
# 使用自定義關鍵詞生成圖片
result = generator.generate_images(
    keywords=["櫻花", "春天", "溫暖的陽光"],
    system_prompt="stable_diffusion_prompt",
    character="kirby",
    num_images=4,
    output_subdir="text2image_custom"
)

print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
for i, img_path in enumerate(result['media_files'], 1):
    print(f"   圖片 {i}: {img_path}")


2025-12-07 23:04:19,983 - examples.simple_content_service - INFO - 開始簡化內容生成流程（跳過分析和文章生成）
2025-12-07 23:04:19,988 - examples.simple_content_service - INFO - 使用策略: text2img
2025-12-07 23:04:19,990 - examples.simple_content_service - INFO - 策略配置載入完成
2025-12-07 23:04:19,991 - examples.simple_content_service - INFO - 開始生成描述
2025-12-07 23:04:19,991 - examples.simple_content_service - INFO - 採用圖片生成策略 : stable_diffusion_prompt



🎨 開始生成圖片...
🔖 Keywords: 櫻花, 春天, 溫暖的陽光
📝 System Prompt: stable_diffusion_prompt
👤 Character: kirby
📊 數量: 4
Using image system prompt key: stable_diffusion_prompt


2025-12-07 23:06:02,496 - examples.simple_content_service - INFO - 描述生成完成，共 1 個描述
2025-12-07 23:06:02,498 - examples.simple_content_service - INFO - 描述 1: Spring landscape with cherry blossom trees in full bloom, warm golden hour sunlight with volumetric rays, Kirby the pink round character standing happily among falling petals, soft focus background with impasto texture
2025-12-07 23:06:02,498 - examples.simple_content_service - INFO - 開始生成圖片


Image descriptions : ['Spring landscape with cherry blossom trees in full bloom, warm golden hour sunlight with volumetric rays, Kirby the pink round character standing happily among falling petals, soft focus background with impasto texture']
生成描述花費 : 102.50 秒
工作流已提交，prompt_id: 89df67e6-a148-45b6-8340-42b1e5c559d2
開始等待工作流 89df67e6-a148-45b6-8340-42b1e5c559d2 完成...
  → 佇列中還有 1 個任務
  → 佇列中還有 1 個任務
  → 正在處理節點: 182
  → 正在處理節點: 260:255
  → 正在處理節點: 260:257
  → 正在處理節點: 260:258
  → 正在處理節點: 260:259
  → 正在處理節點: 269:264
  → 正在處理節點: 269:266
  → 正在處理節點: 269:267
  → 正在處理節點: 269:268
  → 正在處理節點: 272:264
  → 正在處理節點: 272:266
  → 正在處理節點: 272:267
  → 正在處理節點: 272:268
✓ 工作流 89df67e6-a148-45b6-8340-42b1e5c559d2 執行完成（耗時 68.78 秒）
工作流 89df67e6-a148-45b6-8340-42b1e5c559d2 執行完成
工作流已提交，prompt_id: 5dc8948d-11c7-4503-92ca-f47bd7ca7a9e
開始等待工作流 5dc8948d-11c7-4503-92ca-f47bd7ca7a9e 完成...
  → 佇列中還有 1 個任務
  → 佇列中還有 1 個任務
  → 正在處理節點: 260:257
  → 正在處理節點: 260:258
  → 正在處理節點: 260:259
  → 正在處理節點: 269:266
  → 正在處理節點: 269:267


2025-12-07 23:10:02,152 - examples.simple_content_service - INFO - 圖片生成完成，共生成 12 張圖片
2025-12-07 23:10:02,155 - examples.simple_content_service - INFO - 跳過圖文匹配分析（範例模式）
2025-12-07 23:10:02,155 - examples.simple_content_service - INFO - 跳過文章內容生成（範例模式）


✓ 工作流 966467fb-6680-413a-9d6d-9d2053e614a2 執行完成（耗時 55.86 秒）
工作流 966467fb-6680-413a-9d6d-9d2053e614a2 執行完成
✅ 生成圖片總耗時: 239.65 秒
✅ 完成！生成了 12 張圖片
📂 保存位置: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\text2image_custom

✅ 生成完成！共 12 張圖片
   圖片 1: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\text2image_custom\model0_00001__kirby_d0_0.png
   圖片 2: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\text2image_custom\model0_00002__kirby_d0_1.png
   圖片 3: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\text2image_custom\model0_00003__kirby_d0_2.png
   圖片 4: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\text2image_custom\model0_00004__kirby_d0_3.png
   圖片 5: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\text2image_custom\model1_00001__kirby_d0_0.png
   圖片 6: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\text2image_custom\model1_00002__kirby_d0_1.png
   

## 1.2 從資料庫獲取 News 範例


In [ ]:
# 從資料庫獲取新聞並生成圖片
news = get_news_from_db(days_back=7)

if news:
    result = generator.generate_images(
        keywords=news['keyword'] or news['title'],
        system_prompt="stable_diffusion_prompt",
        character="kirby",
        num_images=4,
        output_subdir="text2image_news"
    )
    
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
    for i, img_path in enumerate(result['media_files'], 1):
        print(f"   圖片 {i}: {img_path}")


## 1.3 批量生成範例（生成 30 張圖片）


In [ ]:
# 批量生成 30 張圖片（使用自定義關鍵詞）
results = batch_generate_by_count(
    strategy_type='text2image',
    num_total=30,
    use_news=False,
    custom_keywords="peaceful scene, beautiful landscape",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_images=4  # 每批次生成 4 張
)

# 顯示結果
total_generated = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_generated} 張圖片")


In [ ]:
# 批量生成 50 張圖片（使用資料庫新聞）
results = batch_generate_by_count(
    strategy_type='text2image',
    num_total=50,
    use_news=True,  # 從資料庫獲取新聞
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_images=4
)

# 顯示結果
total_generated = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_generated} 張圖片")


# 2. Image2ImageStrategy - 圖生圖

基於現有圖片進行風格轉換或修改。


## 2.1 自定義 Prompt 範例

⚠️ 注意：需要先有一張輸入圖片


In [ ]:
# 先找一張現有的圖片作為輸入
input_image = None
for pattern in ['output_media/**/*.png', 'output_media/*.png']:
    images = glob.glob(pattern, recursive=True)
    if images:
        input_image = images[0]
        break

if input_image:
    print(f"📷 使用輸入圖片: {input_image}")
    
    config = build_config_for_strategy(
        strategy_type='image2image',
        keywords="watercolor style, artistic",
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        input_image_path=input_image,
        denoise=0.6,  # 0.5-0.7 之間，值越小越接近原圖
        workflow='image_to_image.json'
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
else:
    print("⚠️ 未找到輸入圖片，請先運行 text2image 生成一些圖片")


## 2.2 從資料庫獲取 News 範例


In [ ]:
# 從資料庫獲取新聞作為提示詞
news = get_news_from_db()

if news and input_image:
    config = build_config_for_strategy(
        strategy_type='image2image',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        input_image_path=input_image,
        denoise=0.6,
        workflow='image_to_image.json'
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
else:
    print("⚠️ 需要新聞和輸入圖片才能執行")


# 3. Text2Image2ImageStrategy - 文生圖 → 圖生圖

兩階段生成：先從文字生成圖片，再對圖片進行風格轉換。


## 3.1 自定義 Prompt 範例


In [ ]:
# 文生圖 → 圖生圖
config = build_config_for_strategy(
    strategy_type='text2image2image',
    keywords="sunset, ocean, peaceful",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_images=2,  # 第一階段生成 2 張
    images_per_input=2,  # 第二階段每張生成 2 張
    workflow='nova-anime-xl.json'
)

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")


## 3.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='text2image2image',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        num_images=2,
        images_per_input=2,
        workflow='nova-anime-xl.json'
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")


# 4. Text2VideoStrategy - 文生影片

直接從文字描述生成影片。


## 4.1 自定義 Prompt 範例


In [ ]:
# 生成影片
result = generator.generate_videos(
    keywords=["flying", "stars", "night sky", "peaceful"],
    system_prompt="stable_diffusion_prompt",
    character="kirby",
    num_videos=2,
    output_subdir="text2video_custom"
)

print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")
for i, video_path in enumerate(result['media_files'], 1):
    print(f"   影片 {i}: {video_path}")


## 4.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成影片
news = get_news_from_db()

if news:
    result = generator.generate_videos(
        keywords=news['keyword'] or news['title'],
        system_prompt="stable_diffusion_prompt",
        character="kirby",
        num_videos=2,
        output_subdir="text2video_news"
    )
    
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")


## 4.3 批量生成範例（生成 10 個影片）


In [ ]:
# 批量生成 10 個影片
results = batch_generate_by_count(
    strategy_type='text2video',
    num_total=10,
    use_news=True,  # 從資料庫獲取新聞
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_videos=2  # 每批次生成 2 個
)

total_generated = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_generated} 個影片")


# 5. Text2Image2VideoStrategy - 文生圖 → 圖生影片

兩階段生成：先從文字生成圖片，再將圖片轉換為影片。


## 5.1 自定義 Prompt 範例


In [ ]:
# 文生圖 → 圖生影片
result = generator.generate_text2image2video(
    keywords=["cinematic shot", "dramatic lighting", "slow motion"],
    character="kirby",
    style="movie style",
    num_images=1,  # 第一階段生成 1 張圖片
    num_videos_per_image=1,  # 第二階段每張圖片生成 1 個影片
    output_subdir="text2image2video_custom"
)

print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")
for i, video_path in enumerate(result['media_files'], 1):
    print(f"   影片 {i}: {video_path}")


## 5.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成
news = get_news_from_db()

if news:
    result = generator.generate_text2image2video(
        keywords=news['keyword'] or news['title'],
        character="kirby",
        num_images=1,
        num_videos_per_image=1,
        output_subdir="text2image2video_news"
    )
    
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")


# 6. Text2LongVideoStrategy - 文生長影片（尾幀驅動）

生成長影片，使用尾幀作為下一段的起始。


## 6.1 自定義 Prompt 範例


In [ ]:
# 文生長影片（尾幀驅動）
config = build_config_for_strategy(
    strategy_type='text2longvideo',
    keywords="adventure story, journey through different landscapes",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    workflow='nova-anime-xl.json'
)

# 添加長影片特定配置
config.additional_params = config.additional_params or {}
config.additional_params['strategies'] = {
    'text2longvideo': {
        'longvideo_config': {
            'segment_count': 5,  # 生成 5 個段落
            'segment_duration': 5,  # 每個段落 5 秒
            'use_tts': False  # 是否使用 TTS
        },
        'first_stage': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
            'batch_size': 3
        },
        'video_generation': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
        }
    }
}

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！共 {len(result.get('media_files', []))} 個影片段落")


## 6.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成長影片
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='text2longvideo',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        workflow='nova-anime-xl.json'
    )
    
    config.additional_params = config.additional_params or {}
    config.additional_params['strategies'] = {
        'text2longvideo': {
            'longvideo_config': {
                'segment_count': 3,
                'segment_duration': 5,
                'use_tts': False
            },
            'first_stage': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
                'batch_size': 2
            },
            'video_generation': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
            }
        }
    }
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！")


# 7. Text2LongVideoFirstFrameStrategy - 文生長影片（首幀驅動）

生成長影片，使用 I2I 轉換首幀以保持風格連貫。


## 7.1 自定義 Prompt 範例


In [ ]:
# 文生長影片（首幀驅動）
config = build_config_for_strategy(
    strategy_type='text2longvideo_firstframe',
    keywords="epic journey, changing scenes, cinematic",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    workflow='nova-anime-xl.json'
)

# 添加首幀驅動特定配置
config.additional_params = config.additional_params or {}
config.additional_params['strategies'] = {
    'text2longvideo_firstframe': {
        'longvideo_config': {
            'segment_count': 4,
            'segment_duration': 5,
            'use_tts': False
        },
        'first_stage': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
            'batch_size': 2
        },
        'frame_transition': {
            'enabled': True,
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'image_to_image.json'),
            'denoise': 0.55  # 保持風格，允許場景變化
        },
        'video_generation': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
        }
    }
}

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！")


## 7.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成長影片（首幀驅動）
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='text2longvideo_firstframe',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        workflow='nova-anime-xl.json'
    )
    
    config.additional_params = config.additional_params or {}
    config.additional_params['strategies'] = {
        'text2longvideo_firstframe': {
            'longvideo_config': {
                'segment_count': 3,
                'segment_duration': 5,
                'use_tts': False
            },
            'first_stage': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
                'batch_size': 2
            },
            'frame_transition': {
                'enabled': True,
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'image_to_image.json'),
                'denoise': 0.55
            },
            'video_generation': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
            }
        }
    }
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！")


# 8. StickerPackStrategy - 貼圖包生成

生成多個表情貼圖，可選生成動畫 GIF。


## 8.1 自定義 Prompt 範例


In [ ]:
# 生成貼圖包
config = build_config_for_strategy(
    strategy_type='sticker_pack',
    keywords="happy, sad, angry, surprised, sleepy",  # 表情關鍵詞
    character="kirby",
    system_prompt="sticker_prompt_system_prompt",  # 貼圖專用提示詞
    workflow='nova-anime-xl.json',
    expressions_count=10,  # 生成 10 個表情
    animated_enabled=True  # 啟用動畫 GIF
)

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！")
print(f"   靜態貼圖: {len(result.get('media_files', []))} 張")

# 注意：動畫 GIF 需要通過 handle_review_result 選擇後生成


## 8.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成貼圖包
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='sticker_pack',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="sticker_prompt_system_prompt",
        workflow='nova-anime-xl.json',
        expressions_count=8,
        animated_enabled=True
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！")


## 8.3 批量生成範例（生成 30 個表情）


In [ ]:
# 批量生成貼圖包
results = []
num_batches = 3  # 生成 3 批次，每批次 10 個表情

for i in range(num_batches):
    print(f"\n[{i+1}/{num_batches}] 生成貼圖包...")
    
    news = get_news_from_db()
    if news:
        config = build_config_for_strategy(
            strategy_type='sticker_pack',
            keywords=news['keyword'] or news['title'],
            character="kirby",
            system_prompt="sticker_prompt_system_prompt",
            workflow='nova-anime-xl.json',
            expressions_count=10,
            animated_enabled=False  # 批量生成時關閉動畫以加快速度
        )
        
        result = content_service.generate_content(config)
        results.append({
            'batch': i+1,
            'result': result
        })
        
        print(f"✅ 批次 {i+1} 完成")

total_stickers = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_stickers} 個貼圖")


# 總結

本 Notebook 展示了所有策略的使用方法：

1. ✅ **Text2ImageStrategy** - 文生圖
2. ✅ **Image2ImageStrategy** - 圖生圖  
3. ✅ **Text2Image2ImageStrategy** - 文生圖 → 圖生圖
4. ✅ **Text2VideoStrategy** - 文生影片
5. ✅ **Text2Image2VideoStrategy** - 文生圖 → 圖生影片
6. ✅ **Text2LongVideoStrategy** - 文生長影片（尾幀驅動）
7. ✅ **Text2LongVideoFirstFrameStrategy** - 文生長影片（首幀驅動）
8. ✅ **StickerPackStrategy** - 貼圖包生成

每個策略都支援：
- ✅ 自定義 Prompt
- ✅ 從資料庫獲取 News
- ✅ 批量生成（可指定數量）

## 使用提示

1. **批量生成**：使用 `batch_generate_by_count()` 函數可以輕鬆生成指定數量的媒體
2. **新聞獲取**：使用 `get_news_from_db()` 可以從資料庫獲取新聞作為生成素材
3. **策略選擇**：根據需求選擇合適的策略，例如：
   - 簡單圖片生成 → Text2ImageStrategy
   - 風格轉換 → Image2ImageStrategy
   - 高品質影片 → Text2Image2VideoStrategy
   - 長影片 → Text2LongVideoStrategy 或 Text2LongVideoFirstFrameStrategy
   - 表情貼圖 → StickerPackStrategy
